In [1]:
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
import pandas as pd
import numpy as np
from tqdm import tqdm

from DeepPurpose import utils # , dataset, CompoundPred
from DeepPurpose import DTI as models
from DeepPurpose.utils import *
from DeepPurpose.dataset import *
from rdkit import Chem
from rdkit import DataStructs
#from rdkit.ML.Cluster import Butina
#from rdkit.Chem import Draw
#from rdkit.Chem import rdFingerprintGenerator
#from rdkit.Chem.Draw import SimilarityMaps
import warnings
import numpy as np
import random 
import pandas as pd 
warnings.filterwarnings("ignore")

sdf_file = "all_ligands.sdf"
supplier = Chem.SDMolSupplier(sdf_file, sanitize=False)
# morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048)

# Iterate through the molecules and extract SMILES
ligand_id_to_smile_dict = dict()
for mol in supplier:
    if mol is not None:  # Check if molecule is valid
        try:
            # Attempt to sanitize the molecule
            Chem.SanitizeMol(mol)
            # Generate the fingerprint
            smiles = Chem.MolToSmiles(mol)
            id = int(mol.GetProp('GtoPdb_ID'))
            ligand_id_to_smile_dict[id] = smiles
        except Exception as e:
            print(f"Error processing molecule: {e}")

[01:03:23] ERROR: Counts line too short: '' on line4
[01:03:23] ERROR: moving to the beginning of the next molecule
[01:03:24] Explicit valence for atom # 1 N, 4, is greater than permitted


Error processing molecule: Explicit valence for atom # 1 N, 4, is greater than permitted


[01:03:24] Explicit valence for atom # 8 C, 5, is greater than permitted


Error processing molecule: Explicit valence for atom # 8 C, 5, is greater than permitted


[01:03:26] Explicit valence for atom # 0 N, 4, is greater than permitted


Error processing molecule: Explicit valence for atom # 0 N, 4, is greater than permitted


[01:03:27] Explicit valence for atom # 0 O, 3, is greater than permitted


Error processing molecule: Explicit valence for atom # 0 O, 3, is greater than permitted


[01:03:28] Explicit valence for atom # 15 N, 4, is greater than permitted


Error processing molecule: Explicit valence for atom # 15 N, 4, is greater than permitted


In [2]:
dti_df = pd.read_csv('catalytic_receptor_interactions.csv', skiprows=1)
uni_to_aac_df = pd.read_csv('GtP_to_AAC_mapping.csv')


dti_df = dti_df[['Target ID', 'Ligand ID', 'Affinity Median']].dropna()
target_ids = dti_df['Target ID'].to_numpy()
ligand_ids = dti_df['Ligand ID'].to_numpy()
ys = dti_df['Affinity Median'].to_numpy()


uni_to_aac_df = uni_to_aac_df[['GtoPdb IUPHAR ID', 'AAC']]
target_id_to_aac_dict = uni_to_aac_df.groupby("GtoPdb IUPHAR ID")["AAC"].first().to_dict()


ligand_smiles_list = []
target_aac_list = []
affinity_list = []

errors = 0
for j in range(len(target_ids)):
    target_id = int(target_ids[j])
    ligand_id = int(ligand_ids[j])
    if ligand_id in ligand_id_to_smile_dict.keys() and \
        target_id in target_id_to_aac_dict.keys():
        ligand_smiles = ligand_id_to_smile_dict[ligand_id]
        target_aac = target_id_to_aac_dict[target_id]
        
        if len(ligand_smiles) > 0 and len(target_aac) > 0:
            ligand_smiles_list.append(ligand_smiles)
            target_aac_list.append(target_aac)
            affinity_list.append(ys[j])
    else:
        errors += 1

In [3]:
def get_unique_inds(lst):
    seen = {}
    indices = []
    
    for i, val in enumerate(lst):
        if val not in seen:
            seen[val] = i
            indices.append(i)

    return sorted(indices)


unique_inds = get_unique_inds(ligand_smiles_list)
print(len(unique_inds))
print(len(ligand_smiles_list))

451
1091


In [4]:
unique_target_aac_list = [target_aac_list[i] for i in unique_inds]
unique_ligand_smiles_list = [ligand_smiles_list[i] for i in unique_inds]
unique_affinity_list = [affinity_list[i] for i in unique_inds]

In [5]:
drug_encoding = 'CNN'
target_encoding = 'Transformer'

full_data_df = utils.data_process(unique_ligand_smiles_list, unique_target_aac_list, unique_affinity_list, 
                                  drug_encoding, target_encoding, 
                                  split_method='no_split')

num = len(full_data_df)


# isotopes are not distinguished by tanimoto
further_duplicates = []


similar_mat = np.zeros((num, num))
for i in tqdm(range(num)):
    mol_i = Chem.RDKFingerprint(Chem.MolFromSmiles(full_data_df.iloc[i,0]))
    for j in np.arange(i, num):
        mol_j = Chem.RDKFingerprint(Chem.MolFromSmiles(full_data_df.iloc[j,0]))
        similar_mat[i,j] = DataStructs.TanimotoSimilarity(mol_i, mol_j)

        if i != j and similar_mat[i,j] == 1:
            print("--------------------")
            print(f'ERROR for (i,j) = {(i,int(j))}')
            print(full_data_df.iloc[i,0])
            print(full_data_df.iloc[j,0])
            print("--------------------")
            further_duplicates.append(j)

similar_mat = similar_mat + similar_mat.T - np.eye(similar_mat.shape[0])

similar_mat = similar_mat[np.setdiff1d(np.arange(num), further_duplicates)][:,np.setdiff1d(np.arange(num), further_duplicates)]
np.save('similarityMatrix.npy', similar_mat)


Drug Target Interaction Prediction Mode...
in total: 451 drug-target pairs
encoding drug...
unique drugs: 451
encoding protein...
unique target sequence: 62
splitting dataset...
do not do train/test split on the data for already splitted data


100%|██████████| 451/451 [05:34<00:00,  1.35it/s]


In [6]:
unique_target_aac_list = [unique_target_aac_list[i] for i in np.setdiff1d(np.arange(len(unique_target_aac_list)), further_duplicates)]
unique_ligand_smiles_list = [unique_ligand_smiles_list[i] for i in np.setdiff1d(np.arange(len(unique_ligand_smiles_list)), further_duplicates)]
unique_affinity_list = [unique_affinity_list[i] for i in np.setdiff1d(np.arange(len(unique_affinity_list)), further_duplicates)]



cleaned_df = pd.DataFrame({
    'target': unique_target_aac_list,
    'ligand': unique_ligand_smiles_list,
    'affinity': unique_affinity_list,
})

cleaned_df.to_csv('./cleaned_drug_data.csv')

In [ ]:
for seed in tqdm(range(2500)):
    np.random.seed(seed)

    drug_encoding = 'CNN'
    target_encoding = 'Transformer'


    df = pd.read_csv('cleaned_drug_data.csv')

    affinity_list = df['affinity'].tolist()
    ligand_smiles_list = df['ligand'].tolist()
    target_aac_list = df['target'].tolist()

    n_total = len(affinity_list)
    reind = np.random.permutation(n_total)


    frac = 1./3

    X_drugs_train = [ligand_smiles_list[reind[0:int(n_total*frac+1)][i]] for i in range(int(n_total*frac+1))]
    X_targets_train = [target_aac_list[reind[0:int(n_total*frac+1)][i]] for i in range(int(n_total*frac+1))]
    y_train = [affinity_list[reind[0:int(n_total*frac+1)][i]] for i in range(int(n_total*frac+1))]


    X_drugs_other = [ligand_smiles_list[reind[int(1+n_total*frac):n_total][i]] for i in range(n_total-int(n_total*frac+1))]
    X_targets_other = [target_aac_list[reind[int(1+n_total*frac):n_total][i]] for i in range(n_total-int(n_total*frac+1))]
    y_other = [affinity_list[reind[int(1+n_total*frac):n_total][i]] for i in range(n_total-int(n_total*frac+1))]

    ttrain, tval, ttest = utils.data_process(X_drugs_train, X_targets_train, y_train, 
                                    drug_encoding, target_encoding, 
                                    split_method='random', frac=[1.,0.,0.],
                                    random_seed = seed)

    ddata, _, __ = utils.data_process(X_drugs_other, X_targets_other, y_other, 
                                    drug_encoding, target_encoding, 
                                        split_method='random', frac=[1., 0., 0.],
                                        random_seed = seed)


    calib_test_perm = np.random.permutation(len(ddata))
    final_perm = np.array(reind[int(1+n_total*frac):n_total])[calib_test_perm]
    np.save(f'./permutations/perm_j{seed}.npy', final_perm)


  0%|          | 3/2500 [00:00<03:09, 13.17it/s]

Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 37
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 54
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 37
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 57
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 39
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encod

  0%|          | 5/2500 [00:00<02:39, 15.65it/s]

unique target sequence: 41
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 53
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 34
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 52
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 39
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 47
splitting dataset...
Done.
Drug Target Interaction Pr

  0%|          | 11/2500 [00:00<02:01, 20.47it/s]

...

splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 40
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 48
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 42
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 51
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 38
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 

100%|█████████▉| 2498/2500 [01:41<00:00, 23.82it/s]

splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 37
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 53
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 40
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 52
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 37
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 

100%|██████████| 2500/2500 [01:41<00:00, 24.63it/s]

splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 151 drug-target pairs
encoding drug...
unique drugs: 151
encoding protein...
unique target sequence: 38
splitting dataset...
Done.
Drug Target Interaction Prediction Mode...
in total: 300 drug-target pairs
encoding drug...
unique drugs: 300
encoding protein...
unique target sequence: 53
splitting dataset...
Done.
